# SCRAPER FOR MAYOCLINIC SITE 


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from bs4 import NavigableString
import time
import pickle

In [2]:
base_URL = 'https://www.mayoclinic.org/diseases-conditions/index?letter='
root = 'https://www.mayoclinic.org'
letters = [chr(i) for i in range(65,91)]
letters.append('0')
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
           "accept-language": "en-US,en;q=0.9,bn-IN;q=0.8,bn;q=0.7,hi;q=0.6,lb;q=0.5",
           "accept-encoding": "gzip, deflate, br"
          }

disease_URLs = set()
data = []
page_counter = 0

In [3]:
def inside_text(cur, end1, end2='', end3='', end4='', end5=''):
    while cur and cur != end1 and cur != end2 and cur != end3 and cur != end4 and cur != end5:
        if isinstance(cur, NavigableString):
            text = cur.strip()
            if len(text):
                yield text
        cur = cur.next_element

In [4]:
for letter in letters:
    new_links_on_page_count = 0
    total_number_of_links_on_page = 0
    URL = base_URL + letter
    print("URL: ", URL)
    page_found = False
    retry_count = 0
    
    while not page_found:
        retry_count += 1
        print("trying for the ",retry_count," number of time(s)")
        page = requests.get(URL, headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        response = soup.find(id='index')
        if response:
            page_found = True
        time.sleep(20)
        
    
    for list_element in response.find_all('li'):
        d_href = list_element.a['href'].strip()
        if d_href.startswith('/diseases'):
            total_number_of_links_on_page += 1
        if d_href.startswith('/diseases') and d_href not in disease_URLs:
            new_links_on_page_count+= 1
            #print(ii)
            print(d_href)
            print(list_element.text.strip())
            print(list_element.a.text)
            print('\n\n')
            s_name = list_element.text.strip()
            loc = s_name.find('(') #location of starting brace
            if loc != -1:
                s_name = s_name[:loc].strip()
            item = {}
            item['link'] = root + d_href
            disease_URLs.add(d_href)

            item['primary name'] = list_element.a.text
            item['secondary name'] = s_name
            disease_URL = item['link']
            disease_page = requests.get(disease_URL, headers=headers)

            d_soup = BeautifulSoup(disease_page.content, 'html.parser')
            if d_soup.find('h2', text='Symptoms'):
                item['symptoms'] = [text for text in inside_text(
                    d_soup.find('h2', text='Symptoms').next_sibling,
                    d_soup.find('h4', text='When to see a doctor'),
                    d_soup.find('h3', text='When to see a doctor'),
                    d_soup.find('h4', text='When to see a doctor'),
                    d_soup.find('h5', text='When to see a doctor'),
                    d_soup.find('h6', text='When to see a doctor')
                )]
                # print ('----'.join(text for text in between(d_soup.find('h2', text='Symptoms').next_sibling,
                #                                 d_soup.find('h4', text='When to see a doctor')))
                #       )
            else:
                item['symptoms'] = ''
            data.append(item)
            time.sleep(1)
        
        page_counter += 1
    time.sleep(10)
    print("number of new links on page: ", new_links_on_page_count)
    print("total number of links on page: ", total_number_of_links_on_page)
        


URL:  https://www.mayoclinic.org/diseases-conditions/index?letter=A
trying for the  1  number of time(s)
/diseases-conditions/atrial-fibrillation/symptoms-causes/syc-20350624
A fib (See: A fib, also known asAtrial fibrillation)
A fib, also known asAtrial fibrillation



/diseases-conditions/abdominal-aortic-aneurysm/symptoms-causes/syc-20350688
Abdominal aortic aneurysm
Abdominal aortic aneurysm



/diseases-conditions/hyperhidrosis/symptoms-causes/syc-20367152
Abnormally excessive sweating (See: Abnormally excessive sweating, also known asHyperhidrosis)
Abnormally excessive sweating, also known asHyperhidrosis



/diseases-conditions/bartholin-cyst/symptoms-causes/syc-20369976
Abscess, Bartholin's (See: Abscess, Bartholin's, also known asBartholin's cyst)
Abscess, Bartholin's, also known asBartholin's cyst



/diseases-conditions/petit-mal-seizure/symptoms-causes/syc-20359683
Absence seizure
Absence seizure



/diseases-conditions/acanthosis-nigricans/symptoms-causes/syc-20368983
Acan

In [6]:
print("number of pages scraped: ", page_counter)

number of pages scraped:  2177


In [10]:
data_df = pd.DataFrame(data)

In [16]:
data_df

,link,primary name,secondary name,symptoms
0,https://www.mayoclinic.org/diseases-conditions...,"A fib, also known asAtrial fibrillation",A fib,[Some people with atrial fibrillation (A-fib) ...
1,https://www.mayoclinic.org/diseases-conditions...,Abdominal aortic aneurysm,Abdominal aortic aneurysm,[Abdominal aortic aneurysms often grow slowly ...
2,https://www.mayoclinic.org/diseases-conditions...,"Abnormally excessive sweating, also known asHy...",Abnormally excessive sweating,[The main symptom of hyperhidrosis is heavy sw...
3,https://www.mayoclinic.org/diseases-conditions...,"Abscess, Bartholin's, also known asBartholin's...","Abscess, Bartholin's","[If you have a small, noninfected Bartholin's ..."
4,https://www.mayoclinic.org/diseases-conditions...,Absence seizure,Absence seizure,[An indication of simple absence seizure is a ...
...,...,...,...,...
1141,https://www.mayoclinic.org/diseases-conditions...,Wrinkles,Wrinkles,"[googleoff: all, Deep, coarse wrinkles, Open p..."
1142,https://www.mayoclinic.org/diseases-conditions...,X-linked agammaglobulinemia,X-linked agammaglobulinemia,"[Babies with, XLA, generally appear healthy fo..."
1143,https://www.mayoclinic.org/diseases-conditions...,Yips,Yips,[The most common symptom associated with the y...
1144,https://www.mayoclinic.org/diseases-conditions...,Zika virus,Zika virus,[As many as 4 out of 5 people infected with th...


In [18]:
with open("disease_urls.pickle", "wb") as fp:
    pickle.dump(disease_URLs, fp)
    
with open("disease_url_data.pickle", "wb") as fp:
    pickle.dump(data, fp)

In [20]:
with open("disease_urls.pickle", "rb") as fp:
    disease_URLs = pickle.load(fp)
    
with open("disease_url_data.pickle", "rb") as fp:
    data = pickle.load(fp)
    
root = 'https://www.mayoclinic.org'

In [21]:
symptoms_data = []
pages_crawled = set()

In [22]:
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
           "accept-language": "en-US,en;q=0.9,bn-IN;q=0.8,bn;q=0.7,hi;q=0.6,lb;q=0.5",
           "accept-encoding": "gzip, deflate, br",
           "sec-fetch-dest": "document",
           "sec-fetch-mode": "navigate",
           "sec-fetch-site": "none",
           "sec-fetch-user": "?1",
           "upgrade-insecure-requests": "1"
          }

In [24]:
null_symptom_counter = 0
crawled_count = len(pages_crawled)
for url in disease_URLs:
    
    if url in pages_crawled:
        continue
    pages_crawled.add(url)
    item = {}
    item['link'] = url
    disease_URL = root + url
    disease_page = requests.get(disease_URL, headers=headers)
    d_soup = BeautifulSoup(disease_page.content, 'html.parser')
    text = ''
    if d_soup.find('h2', text='Symptoms'):
        text = [text for text in inside_text(
            d_soup.find('h2', text='Symptoms').next_sibling,
            [d_soup.find('h4', text='When to see a doctor'),
            d_soup.find('h3', text='When to see a doctor'),
            d_soup.find('h4', text='When to see a doctor'),
            d_soup.find('h5', text='When to see a doctor'),
            d_soup.find('h6', text='When to see a doctor')]
        )]
    if text is '':
        null_symptom_counter += 1
    item["symptoms"] = text
    symptoms_data.append(item)
    crawled_count += 1
    print("number of pages crawled: ", crawled_count)
    
    time.sleep(3)


<>:23: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:23: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/zv/n4b0w4hd759g3cfc4fcllmwh0000gn/T/ipykernel_61933/3116054724.py:23: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if text is '':


number of pages crawled:  2
number of pages crawled:  3
number of pages crawled:  4
number of pages crawled:  5
number of pages crawled:  6
number of pages crawled:  7
number of pages crawled:  8
number of pages crawled:  9
number of pages crawled:  10
number of pages crawled:  11
number of pages crawled:  12
number of pages crawled:  13
number of pages crawled:  14
number of pages crawled:  15
number of pages crawled:  16
number of pages crawled:  17
number of pages crawled:  18
number of pages crawled:  19
number of pages crawled:  20
number of pages crawled:  21
number of pages crawled:  22
number of pages crawled:  23
number of pages crawled:  24
number of pages crawled:  25
number of pages crawled:  26
number of pages crawled:  27
number of pages crawled:  28
number of pages crawled:  29
number of pages crawled:  30
number of pages crawled:  31
number of pages crawled:  32
number of pages crawled:  33
number of pages crawled:  34
number of pages crawled:  35
number of pages crawle

In [25]:
with open("symptoms_data.pickle", "wb") as fp:
    pickle.dump(symptoms_data, fp)

In [26]:
symptoms_df = pd.DataFrame(symptoms_data)

In [27]:
symptoms_df.head()

,link,symptoms
0,/diseases-conditions/renal-artery-stenosis/sym...,[Renal artery stenosis often doesn't cause any...
1,/diseases-conditions/bells-palsy/symptoms-caus...,"[googleoff: all, Facial paralysis, Open pop-up..."
2,/diseases-conditions/peanut-allergy/symptoms-c...,[An allergic response to peanuts usually occur...
3,/diseases-conditions/astrocytoma/cdc-20350132,
4,/diseases-conditions/bundle-branch-block/sympt...,"[In most people, bundle branch block doesn't c..."


In [28]:
symptoms_df.to_csv('symptoms_data.csv')
    